# **EEG Brain Age – Random Forest + BAG  (Final 2025)**

### Librerias

In [7]:
import os
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib
matplotlib.use("Agg")  
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, RepeatedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.inspection import permutation_importance
import optuna
import warnings
warnings.filterwarnings("ignore")

### Load Dataset

In [14]:
DATA_PATH = r"C:\Users\Ale\Downloads\Jade\Ale\Data_Parametrizada\EEG_features_final.csv"
SAVE_DIR = r"C:\Users\Ale\Downloads\Jade\Ale\Jade_Saves\Results_RF_Bayessian"
os.makedirs(SAVE_DIR, exist_ok=True)

df = pd.read_csv(DATA_PATH)
df = df.select_dtypes(include=[np.number])
print(f" Dataset loaded: {df.shape[0]} samples, {df.shape[1]} columns")

 Dataset loaded: 510 samples, 10 columns


## **Feature engineering**
Feature engineering consiste en crear nuevas variables a partir de las bandas EEG para representar mejor la actividad cerebral. En este caso, se usan sumas, razones entre bandas y transformaciones matemáticas para capturar relaciones neurofisiológicas relevantes, reducir ruido y facilitar el aprendizaje de los modelos.

In [3]:
if all(col in df.columns for col in ["Delta","Theta","Alpha","Beta","Gamma"]):
    df["TotalPower"] = df[["Delta","Theta","Alpha","Beta","Gamma"]].sum(axis=1)
    df["Theta_Alpha"] = df["Theta"] / (df["Alpha"] + 1e-6)
    df["Alpha_Beta"]  = df["Alpha"] / (df["Beta"] + 1e-6)
    df["Delta_Alpha"] = df["Delta"] / (df["Alpha"] + 1e-6)
    df["Beta_Gamma"]  = df["Beta"]  / (df["Gamma"] + 1e-6)
    df["Slow_Fast"]   = (df["Delta"] + df["Theta"]) / (df["Alpha"] + df["Beta"] + 1e-6)

    for col in ["Delta","Theta","Alpha","Beta","Gamma"]:
        df[f"log_{col}"] = np.log1p(df[col])
        df[f"sqrt_{col}"] = np.sqrt(df[col])

X = df.drop(columns=["Age"])
y = df["Age"]

Z-score filtering was applied to remove outliers, features were standardized to ensure equal scaling, and balanced sample weights were used to reduce bias caused by uneven target distribution. 

In [4]:
z_scores = np.abs(stats.zscore(X))
filtered_entries = (z_scores < 3).all(axis=1)
X = X[filtered_entries].reset_index(drop=True)
y = y[filtered_entries].reset_index(drop=True)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Balanced sample weights
bins = pd.cut(y, bins=5, labels=False)
weights = 1 / np.bincount(bins)
sample_weights = np.array([weights[b] for b in bins])

# **Training MLP**

## **Bayesian hyperparameter optimization using Optuna**

In [5]:
def objective(trial):
    rf = RandomForestRegressor(
        n_estimators=trial.suggest_int("n_estimators", 500, 2000),
        max_depth=trial.suggest_int("max_depth", 10, 80),
        max_features=trial.suggest_categorical("max_features", ["sqrt", "log2"]),
        min_samples_split=trial.suggest_int("min_samples_split", 2, 10),
        min_samples_leaf=trial.suggest_int("min_samples_leaf", 1, 5),
        bootstrap=True,
        random_state=42,
        n_jobs=-1
    )

    cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=42)
    scores = []
    for train_idx, test_idx in cv.split(X_scaled, y):
        X_tr, X_te = X_scaled[train_idx], X_scaled[test_idx]
        y_tr, y_te = y.iloc[train_idx], y.iloc[test_idx]
        w_tr = sample_weights[train_idx]
        rf.fit(X_tr, y_tr, sample_weight=w_tr)
        y_pred = rf.predict(X_te)
        scores.append(r2_score(y_te, y_pred))
    return np.mean(scores)

In [8]:
print("\n Running Optuna Optimization (Random Forest + EEG)...")
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=40, show_progress_bar=True)

print("\n[Optuna Results]")
print(f"Best CV R²: {study.best_value:.3f}")
for k, v in study.best_params.items():
    print(f"  - {k}: {v}")

[I 2026-01-06 19:07:14,041] A new study created in memory with name: no-name-9f524b53-290c-4590-ab42-f4871ff18bd3



 Running Optuna Optimization (Random Forest + EEG)...


Best trial: 0. Best value: 0.883202:   2%|▎         | 1/40 [01:58<1:17:16, 118.88s/it]

[I 2026-01-06 19:09:12,939] Trial 0 finished with value: 0.8832024822962707 and parameters: {'n_estimators': 1872, 'max_depth': 63, 'max_features': 'sqrt', 'min_samples_split': 4, 'min_samples_leaf': 2}. Best is trial 0 with value: 0.8832024822962707.


Best trial: 0. Best value: 0.883202:   5%|▌         | 2/40 [03:01<54:10, 85.54s/it]   

...

[I 2026-01-06 19:56:31,950] Trial 37 finished with value: 0.9284103791732808 and parameters: {'n_estimators': 1321, 'max_depth': 17, 'max_features': 'sqrt', 'min_samples_split': 4, 'min_samples_leaf': 1}. Best is trial 36 with value: 0.9403969429264177.


Best trial: 36. Best value: 0.940397:  98%|█████████▊| 39/40 [50:03<01:00, 60.45s/it]

[I 2026-01-06 19:57:17,262] Trial 38 finished with value: 0.88160953804219 and parameters: {'n_estimators': 1533, 'max_depth': 25, 'max_features': 'log2', 'min_samples_split': 3, 'min_samples_leaf': 2}. Best is trial 36 with value: 0.9403969429264177.


Best trial: 36. Best value: 0.940397: 100%|██████████| 40/40 [50:41<00:00, 76.04s/it]

[I 2026-01-06 19:57:55,797] Trial 39 finished with value: 0.9395399085862195 and parameters: {'n_estimators': 1272, 'max_depth': 20, 'max_features': 'sqrt', 'min_samples_split': 2, 'min_samples_leaf': 1}. Best is trial 36 with value: 0.9403969429264177.

[Optuna Results]
Best CV R²: 0.940
  - n_estimators: 1036
  - max_depth: 18
  - max_features: log2
  - min_samples_split: 2
  - min_samples_leaf: 1


### **Train Final Model**

In [15]:
best_params = study.best_params
rf_best = RandomForestRegressor(**best_params, bootstrap=True, random_state=42, n_jobs=-1)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, shuffle=True)
rf_best.fit(X_train, y_train, sample_weight=sample_weights[y_train.index])
y_pred = rf_best.predict(X_test)

In [16]:
mae = mean_absolute_error(y_test, y_pred)
r2  = r2_score(y_test, y_pred)
r   = np.corrcoef(np.array(y_test), np.array(y_pred))[0, 1]

print("\n================= FINAL TEST RESULTS =================")
print(f"MAE (years): {mae:.2f}")
print(f"R²: {r2:.3f}")
print(f"Pearson r: {r:.3f}")


================= FINAL TEST RESULTS =================
MAE (years): 4.46
R²: 0.898
Pearson r: 0.960


### **BAG Calculation and Categorization**

In [18]:
BAG = y_pred - y_test.values
bag_mean, bag_std = BAG.mean(), BAG.std()
print(f"\n Brain Age Gap (BAG): mean={bag_mean:.2f}, std={bag_std:.2f}")

def categorize_bag(bag):
    if bag < -3: return "Resilient"
    elif bag > 3: return "Accelerated"
    else: return "Normal"

bag_categories = np.vectorize(categorize_bag)(BAG)
bag_df = pd.DataFrame({
    "Chronological_Age": y_test.values,
    "Predicted_Age": y_pred,
    "BAG": BAG,
    "Category": bag_categories
})
bag_df.to_csv(os.path.join(SAVE_DIR, "EEG_Brain_Age_Gap_RF.csv"), index=False)


 Brain Age Gap (BAG): mean=-0.61, std=6.73


### Histograms

In [ ]:
# BAG histogram
plt.figure(figsize=(7,5))
plt.hist(BAG, bins=25, color="lightcoral", edgecolor="black")
plt.axvline(0, color="blue", linestyle="--")
plt.title("Brain Age Gap (Predicted - Chronological) – Random Forest")
plt.xlabel("BAG (years)")
plt.ylabel("Count")
plt.grid(alpha=0.4)
plt.tight_layout()
plt.savefig(os.path.join(SAVE_DIR, "bag_hist_RF.png"))
plt.close()

# BAG categories
plt.figure(figsize=(6,4))
pd.Series(bag_categories).value_counts().reindex(["Resilient","Normal","Accelerated"]).plot(kind="bar", color=["#66c2a5","#fc8d62","#8da0cb"])
plt.title("BAG Categories Distribution (±3 years)")
plt.ylabel("Count")
plt.grid(axis="y", alpha=0.4)
plt.tight_layout()
plt.savefig(os.path.join(SAVE_DIR, "bag_categories_RF.png"))
plt.close()



### Feature Importance

In [19]:
feat_imp = pd.DataFrame({
    "Feature": X.columns,
    "Importance": rf_best.feature_importances_
}).sort_values("Importance", ascending=False)

feat_imp.to_csv(os.path.join(SAVE_DIR, "RF_feature_importance.csv"), index=False)
plt.figure(figsize=(8,6))
plt.barh(feat_imp["Feature"][:10], feat_imp["Importance"][:10], color="darkorange")
plt.gca().invert_yaxis()
plt.title("Top 10 Feature Importances – Random Forest")
plt.tight_layout()
plt.savefig(os.path.join(SAVE_DIR, "RF_feature_importance.png"))
plt.close()

### Permutation Importance

In [20]:

print("\n Computing permutation importance...")
perm_result = permutation_importance(
    rf_best, X_test, y_test, n_repeats=10, random_state=42, scoring="r2"
)
perm_df = pd.DataFrame({
    "Feature": X.columns,
    "Importance": perm_result.importances_mean
}).sort_values("Importance", ascending=False)

perm_df.to_csv(os.path.join(SAVE_DIR, "RF_permutation_importance.csv"), index=False)
plt.figure(figsize=(8,6))
plt.barh(perm_df["Feature"][:10], perm_df["Importance"][:10], color="goldenrod")
plt.gca().invert_yaxis()
plt.title("Top 10 Features – Permutation Importance (RF)")
plt.tight_layout()
plt.savefig(os.path.join(SAVE_DIR, "RF_permutation_importance.png"))
plt.close()




 Computing permutation importance...


In [21]:

pd.DataFrame([{
    "MAE": mae, "R2": r2, "r": r, "BAG_mean": bag_mean, "BAG_std": bag_std,
    "Best_Params": best_params
}]).to_csv(os.path.join(SAVE_DIR, "RF_final_results.csv"), index=False)

print(f"\n All Random Forest results saved in: {SAVE_DIR}")


 All Random Forest results saved in: C:\Users\Ale\Downloads\Jade\Ale\Jade_Saves\Results_RF_Bayessian


### Comparison Trials

In [25]:
def BAG_stadistics(y_pred, y_test):
    BAG = y_pred - y_test.values
    bag_mean, bag_std = BAG.mean(), BAG.std()
    print(f"\n Brain Age Gap (BAG): mean={bag_mean:.2f}, std={bag_std:.2f}")
    return bag_mean, bag_std

    
def comparison_trials(n_trials, best_params, X_train, y_train, X_test, y_test):
    results = []
    for i in range(n_trials):
        print("\nTraining final model with best hyperparameters...")
        print("Trial:", i+1)
        
        rf_best = RandomForestRegressor(**best_params, bootstrap=True, random_state=42, n_jobs=-1)

        X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, shuffle=True)
        rf_best.fit(X_train, y_train, sample_weight=sample_weights[y_train.index])
        y_pred = rf_best.predict(X_test)
        
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        r = np.corrcoef(y_test, y_pred)[0, 1]
        bag_mean, bag_std = BAG_stadistics(y_pred, y_test)

        results.append({
    "Trial": i + 1,
    "MAE": mae,
    "R2": r2,
    "Pearson_r": r,
    "BAG_mean": bag_mean,
    "BAG_std": bag_std,
    "Model": "RandomForest",
    "N_estimators": best_params["n_estimators"],
    "Max_depth": best_params["max_depth"]
        })


    # save once at the end
    results_df = pd.DataFrame(results)
    results_df.to_csv(
        os.path.join(SAVE_DIR, "TRIALS_RF_Optuna_Bayessian.csv"),
        index=False
    )

    print(f"\nAll results saved in: {SAVE_DIR}")

In [27]:
import time
star_time = time.time()
n=20
comparison_trials(20, best_params, X_train, y_train, X_test, y_test)
end_time = time.time()
print(f"\n Total comparison trials time: {end_time - star_time:.2f} seconds") 


Training final model with best hyperparameters...
Trial: 1

 Brain Age Gap (BAG): mean=-0.61, std=6.73

Training final model with best hyperparameters...
Trial: 2

 Brain Age Gap (BAG): mean=-0.61, std=6.73

Training final model with best hyperparameters...
Trial: 3

 Brain Age Gap (BAG): mean=-0.61, std=6.73

Training final model with best hyperparameters...
Trial: 4

 Brain Age Gap (BAG): mean=-0.61, std=6.73

Training final model with best hyperparameters...
Trial: 5

 Brain Age Gap (BAG): mean=-0.61, std=6.73

Training final model with best hyperparameters...
Trial: 6

 Brain Age Gap (BAG): mean=-0.61, std=6.73

Training final model with best hyperparameters...
Trial: 7

 Brain Age Gap (BAG): mean=-0.61, std=6.73

Training final model with best hyperparameters...
Trial: 8

 Brain Age Gap (BAG): mean=-0.61, std=6.73

Training final model with best hyperparameters...
Trial: 9

 Brain Age Gap (BAG): mean=-0.61, std=6.73

Training final model with best hyperparameters...
Trial: 10

 B

Trials Time: 32.46 seconds